In [11]:
!pip install boto3

import pyspark
import boto3

from io import StringIO 
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, StringType, IntegerType 
from pyspark.sql.types import ArrayType, DoubleType, BooleanType
from pyspark.sql.functions import col,array_contains

In [12]:
conf = pyspark.SparkConf()

conf.setMaster("local[1]") 
conf.set("spark.driver.host", "awari-jupyterlab") \
    .set("spark.sql.sources.commitProtocolClass", "org.apache.spark.sql.execution.datasources.SQLHadoopMapReduceCommitProtocol") \
    .set("parquet.enable.summary-metadata", "false") \
    .set("mapreduce.fileoutputcommitter.marksuccessfuljobs", "false") \
    .set("spark.driver.port", "20020") \
    .set("spark.hadoop.fs.s3a.endpoint", 'awari-nginx:9000') \
    .set("spark.hadoop.fs.s3a.endpoint.region", 'sa-east-1') \
    .set("spark.hadoop.fs.s3a.access.key", 'pyUuyVY1RvV1zJ4w') \
    .set("spark.hadoop.fs.s3a.secret.key", 'gUxBmkTbVktZcxCCcALKfhNltUkIQyiT') \
    .set("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .set("spark.hadoop.fs.s3a.connection.ssl.enabled", "false") \
    .set("spark.hadoop.com.amazonaws.services.s3.enableV2", "true") \
    .set("spark.hadoop.fs.s3a.committer.staging.conflict-mode", "replace") \
    .set("spark.hadoop.fs.s3a.fast.upload", True) \
    .set("spark.hadoop.fs.s3a.path.style.access", True) \
    .set("spark.hadoop.fs.s3a.committer.name", "directory") \
    .set("spark.hadoop.fs.s3a.committer.staging.tmp.path", "/tmp/staging")

conf.setAppName('AwariAula08-S33')
sc = pyspark.SparkContext(conf=conf)

spark = SparkSession(sc)

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=AwariAula08-S33, master=local[1]) created by __init__ at /tmp/ipykernel_96/1905431943.py:23 

In [4]:
client = boto3.client('s3', 
    endpoint_url='http://awari-minio-nginx:9000',
    aws_access_key_id='pyUuyVY1RvV1zJ4w',
    aws_secret_access_key='gUxBmkTbVktZcxCCcALKfhNltUkIQyiT',
    aws_session_token=None,
    config=boto3.session.Config(signature_version='s3v4'),
    verify=False,
    region_name='sa-east-1'
)

In [5]:
# Importar CSVs da pasta /exercicios/municipios-estados/csv/ e salvar como arquivos JSON no MinIO.
df_estados_csv = spark.read.option("header", True).csv("arquivos/estados.csv")
df_municipios_csv = spark.read.option("header", True).csv("arquivos/municipios.csv")

df_estados_csv.write.format("json").option("header", "true").mode("overwrite").save("s3a://aula-08/meu-json/estados.json")
df_municipios_csv.write.format("json").option("header", "true").mode("overwrite").save("s3a://aula-08/meu-json/municipios.json")

In [19]:
# Importar JSONs da pasta de /exercicios/municipios-estados/csv/ e salvar como arquivos CSV no MinIO.

df_estados_json = spark.read.option("header", True).json("arquivos/estados.json")
df_estados_json.write.option("header", "true").csv("s3a://aula-08/meu-csv/estados.csv")

In [23]:
# Importar CSVs da pasta de /exercicios/municipios-estados/csv/ e salvar como arquivo .parquet no MinIO.

# df_estados_csv.write.save("s3a://aula-08/meu-parquet2")
df_municipios_csv.write.save("s3a://aula-08/meu-parquet3")

In [ ]:
# Importar CSVs da pasta de /exercicios/municipios-estados/csv/ para tabelas no banco de dados PostgreSQL (awari-postgresql-db).
